## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
0,0,Longyearbyen,SJ,78.2186,15.6401,30.04,64,75,3.44,broken clouds
1,1,Kiama,AU,-34.6833,150.8667,68.38,92,30,1.99,scattered clouds
2,2,Pisco,PE,-13.7000,-76.2167,66.25,77,40,9.22,mist
3,3,Port Elizabeth,ZA,-33.9180,25.5701,61.32,83,20,3.44,few clouds
4,4,Busselton,AU,-33.6500,115.3333,58.50,67,85,11.23,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current description
5,5,Rikitea,PF,-23.1203,-134.9692,76.30,68,0,23.64,clear sky
8,8,Hirara,JP,24.8000,125.2833,78.17,85,100,13.13,light rain
15,15,Eirunepe,BR,-6.6603,-69.8736,87.55,65,100,2.77,overcast clouds
19,19,Jalu,LY,29.0331,21.5482,80.02,23,0,12.28,clear sky
21,21,Sambava,MG,-14.2667,50.1667,76.82,70,0,16.82,clear sky


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                252
City                   252
Country                252
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current description    252
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
5,Rikitea,PF,76.30,clear sky,-23.1203,-134.9692,
8,Hirara,JP,78.17,light rain,24.8000,125.2833,
15,Eirunepe,BR,87.55,overcast clouds,-6.6603,-69.8736,
19,Jalu,LY,80.02,clear sky,29.0331,21.5482,
21,Sambava,MG,76.82,clear sky,-14.2667,50.1667,
23,Dingle,PH,78.51,broken clouds,10.9995,122.6711,
27,Dong Hoi,VN,80.64,overcast clouds,17.4833,106.6000,
31,Souillac,MU,75.58,broken clouds,-20.5167,57.5167,
39,Sao Joao Da Barra,BR,80.69,scattered clouds,-21.6403,-41.0511,
40,Jamestown,US,77.95,broken clouds,42.0970,-79.2353,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [34]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.count()

City                   252
Country                252
Max Temp               252
Current description    252
Lat                    252
Lng                    252
Hotel Name             238
dtype: int64

In [35]:
hotel_df.dropna(subset=['Hotel Name'],inplace=True)
hotel_df.count()

City                   238
Country                238
Max Temp               238
Current description    238
Lat                    238
Lng                    238
Hotel Name             238
dtype: int64

In [36]:
hotel_df.head(10)

,City,Country,Max Temp,Current description,Lat,Lng,Hotel Name
5,Rikitea,PF,76.30,clear sky,-23.1203,-134.9692,People ThankYou
8,Hirara,JP,78.17,light rain,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
15,Eirunepe,BR,87.55,overcast clouds,-6.6603,-69.8736,Atrás de você
19,Jalu,LY,80.02,clear sky,29.0331,21.5482,صاحبة الفخامة
21,Sambava,MG,76.82,clear sky,-14.2667,50.1667,Melrose
23,Dingle,PH,78.51,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
27,Dong Hoi,VN,80.64,overcast clouds,17.4833,106.6000,Nam Long Hotel
31,Souillac,MU,75.58,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
39,Sao Joao Da Barra,BR,80.69,scattered clouds,-21.6403,-41.0511,Pousada Porto De Canoas
40,Jamestown,US,77.95,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [38]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig8 = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig8.add_layer(marker_layer)

# 11b. Display the figure
fig8

Figure(layout=FigureLayout(height='420px'))